In [1]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
import subprocess
from docx.shared import Pt
import pandas as pd
import os
import shutil
from tqdm import tqdm

In [2]:
class MailMerge():
    def __init__(self, data ,label):
        self.data = data
        self.label = label

    def row_to_pdf(self):
        for i in self.data.keys():
            if str(self.data.get(i)) == 'nan':
                self.data[i] = ''
                
        f = open('template.docx', 'rb')
        doc = Document(f)
        
        ## General Information
        doc.tables[0].cell(0, 1).text = self.data['doctor_name']
        doc.tables[0].cell(1, 1).text = self.data['doctor_department']
        doc.tables[0].cell(2, 1).text = self.data['doctor_sip']
        doc.tables[0].cell(3, 1).text = self.data['doctor_str']
        doc.tables[0].cell(0, 3).text = self.data['name']
        doc.tables[0].cell(1, 3).text = self.data['gender']
        doc.tables[0].cell(2, 3).text = self.data['dob']
        doc.tables[0].cell(3, 3).text = str(self.data['card_no']).replace('.0', '')
        doc.tables[0].cell(4, 3).text = self.data['payor']
        doc.tables[0].cell(5, 3).text = self.data['corporate']
        
        for i in range(6):
            if i < 4:
                doc.tables[0].cell(i, 1).paragraphs[0].runs[0].font.size = Pt(8)
            doc.tables[0].cell(i, 3).paragraphs[0].runs[0].font.size = Pt(8)
        
        ## complaint & diagnosis
        doc.paragraphs[2].text = self.data['chief_complaints']
        doc.paragraphs[2].runs[0].font.size= Pt(8)
        doc.paragraphs[4].text = str(self.data['diagnosis']) if str(self.data['diagnosis']) != '' else '' + (', ' + str(self.data['suggestion'])) if str(self.data['suggestion']) != '' else '' 
        doc.paragraphs[4].runs[0].font.size= Pt(8)
        
        doc.paragraphs[6].text = self.data['icdx']
        doc.paragraphs[6].runs[0].font.size= Pt(8)
        
        ## Consultation Detail
        doc.tables[1].cell(1, 1).text = str(self.data['consult_id']).replace('.0', '')
        doc.tables[1].cell(1, 2).text = str(self.data['claim_id']).replace('.0', '')
        doc.tables[1].cell(1, 3).text = self.data['date'].strftime('%d/%m/%Y') + ' ' + self.data['time']
        doc.tables[1].cell(1, 4).text = 'IDR '+ str(self.data['consult_fee']).replace('.0', '')
        doc.tables[1].cell(1, 7).text = 'IDR '+ str(self.data['consult_fee']).replace('.0', '')
        doc.tables[1].cell(2, 1).text = str(self.data['order_id']).replace('.0', '')
        
        
        doc.tables[1].cell(1, 1).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(1, 2).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(1, 3).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(1, 4).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(1, 7).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(2, 1).paragraphs[0].runs[0].font.size= Pt(8)
        
        for i in range(1,13):
            j = i + 2
            if str(self.data[f'obat_{i}']) != '':
                doc.tables[1].cell(j, 0).text = self.data[f'obat_{i}']
                doc.tables[1].cell(j, 0).paragraphs[0].alignment = 2
                doc.tables[1].cell(j, 4).text = 'IDR '+ str(self.data[f'total_{i}']).replace('.0', '')
                doc.tables[1].cell(j, 5).text = str(self.data[f'jumlah_{i}']).replace('.0', '')
                doc.tables[1].cell(j, 6).text = self.data[f'unit_obat_{i}']
                doc.tables[1].cell(j, 7).text = 'IDR '+ str(self.data[f'total_{i}']).replace('.0', '')
        
                doc.tables[1].cell(j, 6).paragraphs[0].runs[0].italic = True
                
                doc.tables[1].cell(j, 0).paragraphs[0].runs[0].font.size= Pt(8)
                doc.tables[1].cell(j, 4).paragraphs[0].runs[0].font.size= Pt(8)
                doc.tables[1].cell(j, 5).paragraphs[0].runs[0].font.size= Pt(8)
                doc.tables[1].cell(j, 6).paragraphs[0].runs[0].font.size= Pt(8)
                doc.tables[1].cell(j, 7).paragraphs[0].runs[0].font.size= Pt(8)
        
        doc.tables[1].cell(16, 4).text = 'IDR 0' if self.data['excess_delivery_fee'] =='' else 'IDR '+ str(self.data['excess_delivery_fee']).replace('.0', '')
        doc.tables[1].cell(16, 7).text = 'IDR 0' if self.data['excess_delivery_fee'] =='' else 'IDR '+ str(self.data['excess_delivery_fee']).replace('.0', '')
        doc.tables[1].cell(18, 7).text = 'IDR '+ str(self.data['total']).replace('.0', '')
        excess =  0 if  self.data['rx_excess'] == '' else float(self.data['rx_excess']) + 0 if  self.data['excess_consult'] == '' else self.data['excess_consult']
        doc.tables[1].cell(19, 7).text = 'IDR '+ str(self.data['total'] - excess).replace('.0', '')
        doc.tables[1].cell(20, 7).text = 'IDR '+ str(excess).replace('.0', '')
        
        doc.tables[1].cell(16, 4).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(16, 7).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(18, 7).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(19, 7).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(20, 7).paragraphs[0].runs[0].font.size= Pt(8)
        
        doc.tables[1].cell(18, 7).paragraphs[0].runs[0].bold = True
        doc.tables[1].cell(19, 7).paragraphs[0].runs[0].bold = True
        doc.tables[1].cell(20, 7).paragraphs[0].runs[0].bold = True
        
        name = self.data['name'].replace('/', ' ')
        consult_on = self.data['date'].strftime('%Y-%m-%d')
        consult_id = str(self.data['consult_id']).replace('.0', '')
        agg = self.data['aggregator_name']
        payor = self.data['payor']
        
        if not os.path.exists(f'.tmp/{agg}/{payor}'):
            os.makedirs(f'.tmp/{agg}/{payor}')
        
        tmp_name = f'.tmp/{agg}/{payor}/{consult_on}_Consultation_Receipt_{name}_{consult_id}.docx'
        doc.save(tmp_name)
        f.close()
        
        subprocess.run(['libreoffice', '--convert-to', 'pdf' ,
                        tmp_name, '--outdir', f'.tmp/{agg}/{payor}']
                       ,stdout=subprocess.DEVNULL,
                        stderr=subprocess.DEVNULL
                    )
        os.remove(tmp_name)
        
    def chunks(self, lst, n):
        """Yield successive n-sized chunks from lst."""
        for i in range(0, len(lst), n):
            yield lst[i:i + n] 
        
    def chunk_and_zip(self):
        fls = glob.glob(f'{self.label}/*/*/*')
        unique_agg = np.unique([i.split('/')[1] for i in fls])
        unique_pay = np.unique([i.split('/')[2] for i in fls])
        for i in unique_agg:
            for j in unique_pay:
                ij = [x for x in fls if x.split('/')[1] == i and x.split('/')[2] == j]
                k = 1
                for files in self.chunks(ij, 100):
                    with ZipFile(f'output/{self.label}/{i}_{j}_{k}.zip','w') as zip: 
                        for file_each in files:
                            zip.write(file_each, file_each.split('/')[-1]) 
                    k = k + 1

    def run(self):
        for index, row in tqdm(consult_mrg.iterrows()):
            self.row_to_pdf(row)

        self.chunk_and_zip()

In [3]:

consult = pd.read_excel('Input/04. Report PPU April 2024.xlsx', sheet_name='Discharge Consult')
consult.columns = consult.columns.str.lower().str.strip().str.replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]','', regex = True).str.replace(' ', '_')

consult_mrg = pd.read_excel('Input/sample_consult_file_w_prescription_3.xlsx')

consult_mrg.columns = consult_mrg.columns.str.lower().str.strip().str.replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]','', regex = True).str.replace(' ', '_')
consult_mrg = consult_mrg.merge(consult[['consultation_id','dob','gender']])

FileNotFoundError: [Errno 2] No such file or directory: 'Input/04. Report PPU April 2024.xlsx'

In [ ]:
MailMerge()

In [6]:
os.getcwd()

'/home/jovyan/work/output'